### Contextualização:

Um levantamento foi realizado com o objetivo de investigar a influência de normas culturais e a aplicação da lei no controle da corrupção, focando no comportamento de estacionamento dos diplomatas das Nações Unidas em Manhattan. Até 2002, a imunidade diplomática protegia os diplomatas da ONU de penalidades de estacionamento, deixando suas ações restritas apenas pelas normas culturais. Em 2002, as autoridades adquiriram o direito de confiscar as placas diplomáticas dos infratores.

**Perguntas**:

1. Qual seria a quantidade média esperada de violações de trânsito para um país cujo corpo diplomático seja composto por X1 membros, considerando o período anterior à vigência da lei e cujo índice de corrupção seja igual a X2?

2. Qual seria a quantidade média esperada de violações de trânsito para o mesmo país, porém agora considerando a vigência da lei?

Para isso, serão construídos um <span style="font-family:Comic Sans MS; color:red">modelo preditivo Poisson</span> e um <span style="font-family:Comic Sans MS; color:red">modelo preditivo Binomial Negativo</span> e, por fim, será feita a comparação dos resultados produzidos pelos modelos.

In [1]:
# Instalação dos pacotes

# !pip install pandas
# !pip install numpy
# !pip install -U seaborn
# !pip install matplotlib
# !pip install plotly
# !pip install scipy
# !pip install statsmodels
# !pip install scikit-learn
# !pip install statstests